# Assignment: Data Wrangling

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?

*This paper is about "Data Tidying" which is a small, but important, component of data cleaning. *

  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)


In [2]:
import numpy as np 
import pandas as pd 

#1
#cleaning assuption is that when price is >999 the number includes a "," which changes the feature type.
dfBNB = pd.read_csv("./data/airbnb_hw.csv")
#dfBNB
#print(dfBNB.dtypes, '\n') #seen here price is an object type. (Turn it into int64 to match other numerical feature types)
#price = dfBNB['Price']
#for x in price:
#    if int(x) > 999:
#        print(price.unique) 
#dfClean = dfBNB.copy()
#for x in dfBNB["Price"]:
#    dfClean[x] = x.replace(",", "")
#print(dfClean["Price"].unique())

#more info on "Price"
print("pre-coercion to numeric:")
print(dfBNB["Price"].describe())
#dfBNB["Price"].hist()

#removing , in price values
#dfBNB['Price'].str.replace(",", "") ***THIS was the original cause of the bug creating missing values. Did not assign the replaced string value back to dfBNB["Price"] so pd.to_numeric() was still trying to coerce values with "," to numeric.
dfBNB['Price'] = dfBNB['Price'].str.replace(",", "")

#coercing price into a numeric feature
dfBNB["Price"] = pd.to_numeric(dfBNB["Price"], errors="coerce")

#Missing dummies
#dfBNB['Price'+'_nan'] = dfBNB['Price'].isnull()

#post coercion to numeric statistics
print("post coercion to numeric:")
print(dfBNB["Price"].describe()) #price turned to a float64 here which works

#print(dfBNB["Price"].unique()) 
    
#missing values
print('Total Missings: \n', sum(dfBNB["Price"].isnull()),'\n') #after coercion to numeric 181 values are missing

#investigating missing data:
print("price values over 999")
for x in dfBNB["Price"]:
    if x > 999:
        print(x) #nothing prints so that means all data lost were the price values over 999.

pre-coercion to numeric:
count     30478
unique      511
top         150
freq       1481
Name: Price, dtype: object
post coercion to numeric:
count    30478.000000
mean       163.589737
std        197.785454
min         10.000000
25%         80.000000
50%        125.000000
75%        195.000000
max      10000.000000
Name: Price, dtype: float64
Total Missings: 
 0 

price values over 999
1990
1000
1200
1000
5000
3000
1500
1200
1800
1300
1000
1239
5999
1500
1300
1500
2750
4750
1000
8000
4500
1000
1000
2500
1000
1000
3750
1000
1500
1000
1000
1500
1000
2000
1000
1500
1750
1000
1500
1495
1700
1000
5000
1999
1000
1170
1100
3000
1000
1111
1000
1000
1174
4000
1100
1000
2500
1000
1500
1400
2520
1000
1000
1500
1000
1200
1600
1000
2695
1300
1000
2295
1065
2000
3000
2000
2000
1500
1200
5000
1368
1000
1500
1000
1500
1000
1000
1250
1350
2000
1200
1500
1250
1200
1000
1800
4000
4000
3000
4500
3200
2000
1000
2000
1500
1000
1500
2500
1100
1000
1050
1650
1000
1200
1000
1500
1500
1500
1000
1550
1200
2500


*To clean the Price feature, I first looked at some details about the feature and noticed it was categorized as an "object" type even though it should be a numerical feature. Given the hint, I decided to look at price values greater than 999 and noticed that these values included a "," which was making the feature be casted as an object. I replaced the "," with an empty string to remove this seperator so I could then cast the "Price" feature to a numeric. Intitially I had 181 missing values which were all the price values over 999. Looking at my code again I found my error in my initial replace function, so I adjusted the line and ended with 0 missing values.*

2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.


In [8]:
#analyzing the type feature
df = pd.read_csv("./data/sharks.csv", low_memory=False)
print(df["Type"].value_counts())

#for cleaning, should combine these smaller value counts
#combine invalid, questionable, unconfirmed, unverified, and under investigation as these are all 
#keep unprovoked seperate

Type
Unprovoked             4716
Provoked                593
Invalid                 552
Sea Disaster            239
Watercraft              142
Boat                    109
Boating                  92
Questionable             10
Unconfirmed               1
Unverified                1
Under investigation       1
Boatomg                   1
Name: count, dtype: int64


3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?